### Fitting Naive Bayes to the Training set

posterior = prior occurrances * liklihood / evidence
The "gaussian" and "naive" come from two assumptions present in this likelihood:
1. we assume each feature is uncorrelated from each other. That is, meannfreq is                  independent of sd or median etc.. This is obviously not true, and is a "naive"                assumption.
2. we assume have that the value of the features (e.g. meannfreq of women) are normally          (gaussian) distributed. This means that p(meannfreq∣female) is calculated by inputing          the required parameters into the probability density function of the normal                    distribution.


In [38]:
from sklearn.naive_bayes import GaussianNB
import pandas as pd 
import numpy as np  
import random
from sklearn.model_selection import train_test_split  
from collections import defaultdict

In [39]:
# Data preparation
path = '/Users/Kassi/Desktop/Gender_Recognition_by_Voice/voice.csv'
voice_data = pd.read_csv(path)

In [40]:
voice_data = pd.read_csv(path)

In [41]:
## Calculate Priors

# Number of males
n_male = voice_data['label'][voice_data['label'] == 'male'].count()

# Number of males
n_female = voice_data['label'][voice_data['label'] == 'female'].count()

# Total rows
total_ppl = voice_data['label'].count()

# Number of males divided by the total rows
P_male = n_male/float(total_ppl)

# Number of females divided by the total rows
P_female = n_female/float(total_ppl)

In [42]:
P_female

0.5

In [46]:
# Group the data by gender and calculate the means of each feature
data_means = voice_data.groupby('label').mean()
# View the values
data_means

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
label,,,,,,,,,,,,,,,,,,,,
female,0.191000,0.049142,0.195942,0.165349,0.223184,0.057834,2.984875,24.805224,0.873066,0.344763,0.178541,0.191000,0.169742,0.039429,0.263848,0.929544,0.064987,5.736107,5.671120,0.170073
male,0.170813,0.065110,0.175299,0.115562,0.226346,0.110784,3.295460,48.331698,0.917188,0.471670,0.152022,0.170813,0.115872,0.034175,0.253836,0.728877,0.040307,4.358447,4.318139,0.177430


In [47]:
# Group the data by gender and calculate the variance of each feature
data_variance = voice_data.groupby('label').var()
# View the values
data_variance

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx
label,,,,,,,,,,,,,,,,,,,,
female,0.000898,0.000338,0.001034,0.002477,0.000535,0.001842,9.557091,9539.255910,0.002236,0.032348,0.004512,0.000898,0.000341,0.000477,0.000463,0.332795,0.005291,14.853636,14.869694,0.011586
male,0.000689,0.000089,0.001398,0.001024,0.000578,0.000417,26.370174,26606.809819,0.000837,0.022642,0.007060,0.000689,0.000295,0.000248,0.001296,0.198914,0.002421,9.001710,9.003627,0.016934


In [48]:
classnames = np.array(['female','male'])
classnames[0]

'female'

In [49]:
voices = voice_data.values[:, :-1] 
labels = voice_data.values[:, -1:]
# male -> 1
# female -> 0
from sklearn.preprocessing import LabelEncoder
gender_encoder = LabelEncoder()
labels = gender_encoder.fit_transform(labels)

# randomly shuffle data
voices_tmp = []  
lables_tmp = []  
index_shuf = range(len(voices)) 
random.shuffle(index_shuf) 
for i in index_shuf:  
    voices_tmp.append(voices[i])  
    lables_tmp.append(labels[i])  
voices = np.array(voices_tmp)  
labels = np.array(lables_tmp) 
train_x, test_x, train_y, test_y = train_test_split(voices, labels, test_size=0.2) 

In [50]:
labels
classes

array([0, 1])

In [51]:
classes = np.unique(labels)


In [52]:
# For both the classes 0 and 1, and for each of the 20 features, we have thus saved the mean and the standard in summary. 
# For example, for the class 0 (female), and feature 1 (ie, sd), mean and the standard for this attribute are 0.060910268026077803 and 0.00034668419803097784

def getSummary(data,n_cls):
    data_means = data.groupby('label').mean()
    data_variance = data.groupby('label').var()
    
    cnames = np.array(data.columns.tolist())[:-1] 
    classes = np.unique(labels)
    classnames = np.array(['female','male'])
    summary = {}

    for cls in classes:
        #initializing the dictionary
        summary[cls] = defaultdict(list)
    for cls in classes:
        for j in range(0, n_cls):
            summary[cls][j] += list(np.array([data_means[cnames[j]][data_means.index == classnames[cls]].values[0],
                                          data_variance[cnames[j]][data_variance.index == classnames[cls]].values[0]]))
    print summary  
    print classes
    return summary

In [53]:
getSummary(voice_data,train_x.shape[1])

{0: defaultdict(<type 'list'>, {0: [0.19099973070217632, 0.00089758079929079396], 1: [0.049141826078528024, 0.00033780862697837036], 2: [0.19594239311047099, 0.0010335560438297876], 3: [0.16534914705534454, 0.0024767989702860001], 4: [0.22318359968875567, 0.00053457098398006458], 5: [0.05783445263341138, 0.0018424489803619841], 6: [2.9848747790826993, 9.5570905745333068], 7: [24.805224039775759, 9539.2559103765834], 8: [0.87306574341157717, 0.0022361566064679544], 9: [0.34476288506667707, 0.032347516924330133], 10: [0.1785412240405656, 0.0045124519060728572], 11: [0.19099973070217632, 0.00089758079929079396], 12: [0.16974182917136693, 0.00034078273831658883], 13: [0.039428607658650125, 0.00047719224527028827], 14: [0.26384816427816848, 0.00046347866560820036], 15: [0.92954443622617933, 0.33279503875146149], 16: [0.06498684353298613, 0.0052908937498219143], 17: [5.7361069496231849, 14.853636373742516], 18: [5.6711201060901981, 14.869693912877652], 19: [0.17007277031045853, 0.01158625391

{0: defaultdict(list,
             {0: [0.19099973070217632, 0.00089758079929079396],
              1: [0.049141826078528024, 0.00033780862697837036],
              2: [0.19594239311047099, 0.0010335560438297876],
              3: [0.16534914705534454, 0.0024767989702860001],
              4: [0.22318359968875567, 0.00053457098398006458],
              5: [0.05783445263341138, 0.0018424489803619841],
              6: [2.9848747790826993, 9.5570905745333068],
              7: [24.805224039775759, 9539.2559103765834],
              8: [0.87306574341157717, 0.0022361566064679544],
              9: [0.34476288506667707, 0.032347516924330133],
              10: [0.1785412240405656, 0.0045124519060728572],
              11: [0.19099973070217632, 0.00089758079929079396],
              12: [0.16974182917136693, 0.00034078273831658883],
              13: [0.039428607658650125, 0.00047719224527028827],
              14: [0.26384816427816848, 0.00046347866560820036],
              15: [0.92954443

In [54]:
# Create a function to calculate the probability density of each of the terms of the likelihood 
# We use mean and standard deviation of input values (x) for each class to summarize the distribution
# Probabilities of new x values are calculated using the Gaussian Probability Density Function (PDF).

def p_x_given_y(x, mean_y, variance_y):
    # Input the arguments into a probability density function
    p = 1/(np.sqrt(2*np.pi*variance_y)) * np.exp((-(x-mean_y)**2)/(2*variance_y))
    return p

# We calculate the probability that the value of the random variable will be between x - 0.01 and x + 0.01


In [55]:
p_x_given_y(2, 2, 4)

0.19947114020071635

In [56]:
def calculateClassProbabilities(summaries, inputVector):
    # p(x|y) (e.g. p(meannfreq∣female))
    probabilities = {}
    for classValue, classSummaries in summaries.iteritems():
        if (classValue == 0):
            probabilities[classValue] = P_female
        
        else:
            probabilities[classValue] = P_male
        for i in range(len(classSummaries)):
            mean, var = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= p_x_given_y(x, mean, var)
    return probabilities

In [61]:
calculateClassProbabilities(summary, test_x[10])
# PDF can be greater than 1, but it can't be greater than 1 for a large interval
# because of the non-negativity and integral constraints.

{0: 6.547904295252147e-14, 1: 4.5580160160962091e-11}

In [58]:
# Make a Prediction
import operator
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    #print probabilities
    probabilities = sorted(probabilities.iteritems(), key=operator.itemgetter(1), reverse=True)
    return probabilities[0][0]

In [36]:
probabilities = calculateClassProbabilities(summaries, inputVector)
#print probabilities
probabilities = sorted(probabilities.iteritems(), key=operator.itemgetter(1), reverse=True)
return probabilities[0][0]

NameError: name 'summaries' is not defined

In [16]:
print(test_x[10])
print('Our prediction for test_x[10] is %s'  % (predict(summary, test_x[10])))

[0.182440531475707 0.0826908723499453 0.21594095940959399 0.144132841328413
 0.23686346863468602 0.092730627306273 2.50602933066259 10.818060222584501
 0.9211971774786719 0.531724451397257 0.22859778597785999 0.182440531475707
 0.205320023468065 0.0869565217391304 0.25 0.262152777777778 0.0234375
 0.45703125 0.43359375 0.29214929214929203]
Our prediction for test_x[10] is 0


In [17]:
from sklearn.metrics import classification_report

def GaussianNB_Classifier(test_x,test_y):
    results = []
    for i in range(len(test_x)):
        test_single = test_x[i]
        res = predict(summary, test_single)
        results.append(res)
    # We evaluate the accuracy of the model by calculating a ratio of the total correct predictions out of all predictions made (the classification accuracy)
    correct = 0
    wrong = 0
    for j in range(len(test_y)):
        if(results[j] == test_y[j]):
            correct += 1
        else:
            wrong += 1
 
    print('Correct rate is %s'  % (float(correct)/(correct+wrong)))
    print(classification_report(test_y,results))



In [18]:
print("Use classification reports to evaluate how well Gaussian Naive Bayes performed with all features \n")
print("in-sample accuracy in Gaussian Naive Bayes:")
GaussianNB_Classifier(train_x,train_y)

Use classification reports to evaluate how well Gaussian Naive Bayes performed with all features 

in-sample accuracy in Gaussian Naive Bayes:
Correct rate is 0.807065217391
             precision    recall  f1-score   support

          0       0.84      0.76      0.80       184
          1       0.78      0.85      0.82       184

avg / total       0.81      0.81      0.81       368



In [19]:
print("out-of-sample accuracy in Gaussian Naive Bayes:")
GaussianNB_Classifier(test_x,test_y)

out-of-sample accuracy in Gaussian Naive Bayes:
Correct rate is 0.782608695652
             precision    recall  f1-score   support

          0       0.80      0.76      0.78        46
          1       0.77      0.80      0.79        46

avg / total       0.78      0.78      0.78        92



In [20]:
# We are wondering if there is a big difference in the results 
# when used Sklearn package vs. our own algorithm
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

nb = GaussianNB()
nb.fit(train_x, train_y)
prediction = nb.predict(test_x)

print('NB result: ', accuracy_score(test_y, prediction))

('NB result: ', 0.78260869565217395)


In [59]:
# Is it enough of these two features to make predictions? 
## Random Forest with 2 features ('meanfun', 'IQR')

voice_new_data = voice_data[['IQR','meanfun','label']]
train_new_x = train_x[:,[5,12]]
test_new_x = test_x[:,[5,12]]
summary = getSummary(voice_new_data, train_new_x.shape[1])

{0: defaultdict(<type 'list'>, {0: [0.05783445263341138, 0.0018424489803619841], 1: [0.16974182917136693, 0.00034078273831658883]}), 1: defaultdict(<type 'list'>, {0: [0.11078428922589478, 0.00041675862497546877], 1: [0.11587163938708586, 0.00029511394260779895]})}
[0 1]


In [60]:
print("Use classification reports to evaluate how well Gaussian Naive Bayes performed with only 2 features \n")
print("In-sample accuracy in Gaussian Naive Bayes:")
GaussianNB_Classifier(train_new_x,train_y)

Use classification reports to evaluate how well Gaussian Naive Bayes performed with only 2 features 

In-sample accuracy in Gaussian Naive Bayes:


NameError: name 'GaussianNB_Classifier' is not defined

In [23]:
print("Out-of-sample accuracy in Gaussian Naive Bayes:")
GaussianNB_Classifier(test_new_x,test_y)

Out-of-sample accuracy in Gaussian Naive Bayes:
Correct rate is 0.869565217391
             precision    recall  f1-score   support

          0       0.90      0.83      0.86        46
          1       0.84      0.91      0.87        46

avg / total       0.87      0.87      0.87        92



We did observe some improvement in performance when we implement Gaussian Naive Bayes with 2 features.